In [ ]:
def accumulator_naive_c(accm_op, variables, indent_level=1):
    indent = " " * 4 * indent_level
    
    accm_op.append("\n")
    if len(variables) == 1:
        return variables[0]

    variables_next_ = []
    variables_ = variables[:-1] if len(variables) % 2 == 1 else variables
    for i in range(0, len(variables_), 2):
        accm_op.append(f"{indent}{variables_[i]} = {variables_[i]} + {variables_[i+1]};")
        variables_next_.append(f"{variables_[i]}")
        
    if len(variables) % 2 == 1:
        variables_next_.append(f"{variables[-1]}")
        
    accumulator_naive_c(accm_op, variables_next_, indent_level = indent_level)
    

for unroll in range(1, 8+1, 1):
    with open(f"./small_dgemv_naive_c_implementation_ver0-2_Naive-Unroll-LDX-{unroll}.c", "w") as f:
        # Normal ---------------------------------------------------------------------------------
        declare_tmp_x = "double " + ", ".join([f"tmp_x_{i}" for i in range(unroll)])
        declare_tmp_y = "double " + ", ".join([f"tmp_y_{i}" for i in range(unroll)])

        remain = ""
        if unroll>=2:
            remain = f"""
        double tmp_x;
        for (int64_t k=ldx-k_remain; k<ldx; k++){{
            tmp_x = x[k];
            for (int64_t i=0; i<lda; i++){{
                y[i] += a[i + k*lda] * tmp_x;
            }}
        }}        
            """

        unroll_op = ""
        for j in range(0, unroll, 1):
            unroll_op += f"""
                tmp_y_{j} = a[i + (k+{j})*lda] * tmp_x_{j};"""

        variables = ["tmp"] + [f"tmp_y_{i}" for i in range(unroll)]
        accm_op = []
        accumulator_naive_c(accm_op, variables, indent_level=3)

        load_tmp_x = "; ".join([f"tmp_x_{i} = x[k+{i}]" for i in range(unroll)])
        unroll_op += "\n".join(accm_op)


        code = f"""#include <stdint.h>
    #include <stdio.h>

    void mydgemv_n_ver0_2_unroll{unroll}(double a[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
        for (int64_t i=0; i<lda; i++){{
            y[i] = 0.0;
        }}

        int64_t k_remain = ldx % {unroll};
        {declare_tmp_x};
        {declare_tmp_y};
        double tmp;
        for (int64_t k=0; k<ldx-k_remain; k+={unroll}){{
            {load_tmp_x};
            for (int64_t i=0; i<lda; i++){{
                tmp = y[i];
                {unroll_op}
                y[i] = tmp;
            }}
        }}
        {remain}
    }}
            """
        f.write(code)
        
        # Transposed ---------------------------------------------------------------------------------
        declare_tmp_y = "double " + ", ".join([f"tmp_y_{i}" for i in range(unroll)])

        remain = ""
        if unroll>=2:
            remain = f"""
            for (int64_t k=ldx-k_remain; k<ldx; k++){{
                tmp += a_t[i*ldx + k] * x[k];
            }}        
            """

        unroll_op = ""
        for j in range(0, unroll, 1):
            unroll_op += f"""
                tmp_y_{j} = a_t[i*ldx + k+{j}] * x[k+{j}];"""

        variables = ["tmp"] + [f"tmp_y_{i}" for i in range(unroll)]
        accm_op = []
        accumulator_naive_c(accm_op, variables, indent_level=4)

        init_tmp_x = "; ".join([f"tmp_x_{i} = 0.0" for i in range(unroll)])
        unroll_op += "\n".join(accm_op)

        code = f"""
    void mydgemv_t_ver0_2_unroll{unroll}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
        int64_t k_remain = ldx % {unroll};
        {declare_tmp_y};
        double tmp;
        for (int64_t i=0; i<lda; i+=1){{
            tmp = 0.0;
            for (int64_t k=0; k<ldx-k_remain; k+={unroll}){{
                {unroll_op}
            }}
            {remain}
            y[i] = tmp;
        }}
    }}        
        """
#         print(code)
        f.write(code)
        